In [ ]:
import sys
import os
sys.path.append('..')

from src.utils import load_from_json, count_files_in_folder

from src.builder import TableOfContents, GuideBuilder
from src.pdf import Markdown2PDF

# Generation

In [75]:
concepts_instructs = """
Focus on detailed expert level topics within main subject, avoid general concepts like 'deep learning', 'neural_networks' or 'LLMs'.
Try to cover all possible low-level prompting techniques like Tabular Chain-of-Thoughts. Do not include application or usage examples.
"""

concept_instructs2 = """
List all known variations of prompt engineering techniques, their optimization and validation methods. Do not include application or usage examples.
"""

parts_instructs = """
Put strong emphasis on following issues:
1. Add 3 or 4 parts exclusively about specific prompt generation techniques categories
2. In introduction part you should avoid general concepts about LLMs, deep learning etc.
3. Remember that this is a book for people with some knowledge about AI who want to become experts in prompt engineering field.
4. Suggest at least one part about advanced methods.
5. The main goal is to cover all possible prompting techniques.
"""

parts_instructs = """
During section generation follow this rules:
1. Be completely focued on prompting techniques variations.
2. Avoid repeating similar subjects within different sections.
3. Try to cover all possible prompt engineering techniques and their variations.
4. Remember that this is a book for experts in AI field who want to learn about prompt engineering.
5. Limit the number of parts between 8 to 10. 
6. Each part should cover a different group of prompting techniques.
7. There is no need for general introduction chapter. Start directly with some cluster of prompting techniques.
8. Do not include parts about applications in different fields.
9. You should include prompts evaluation methods as a concept in one of the parts.
10. Provide around 10 concepts in each part.

Once more: Each part should be about a different group of prompting techniques.
Spare at least 6 parts on different prompting variants!

Focus on describing rules of prompting.
Do not prepare sections about their usage in different fields

There will be one more nesting level after this section division, that is why keep concepts very general so they could be divided into more specific sections in the next steps.
"""

parts_instructs="""
Divide the book into parts about:
1. Zero-shot learning, One-shot learning, Few-shot learning and their variations, also add general concepts about prompt engineering.
2. Chain of Thoughts, Tree of Thoughts and other chaining techniques.
3. Schematic prompting including graphs or Tabular Chain of Thoughts
4. Combining different techniques and ensembling.
5. Prompt enchancing like for example adding external knowledge and meta prompting.
6. Evaluation of prompts and bias mitigation.
7. Safety issues.
"""

chapters_instructs = [
"""
Take into account the following:
1. You are writing a book for experts in AI field who want to learn about prompt engineering.
2. Be as detailed as possible whitin provided subject but not exceeding it, especially by concepts covered by other sections.
3. Focus on the current section and avoid duplicating concepts from other sections.
4. Include both basic and advanced concepts in prompt engineering.
5. The main goal is to provide a comprehensive guide about all prompt engineering techniques there are.
6. If you are generating subchapters for some specific prompting technique, try to list all versions of that technique.
7. Ideally each chapter should be about a specific prompting technique or its variation.
8. Do not include parts about applications in different fields.
9. Avoid general paragraphs, treat it like a manual to use something. Focus on main concepts.
10. Try to fit number of proposals between 3 to 6 sections.
11. Avoid adding evaluation methods or metrics for sections dedicated to specific prompting techniques.
""",
"""
Remember that you are focusing on the current section so try not to cover subjects from other parts of the book.
This is a book about all Prompt engineering techniques, from the most basic to the most advanced, so use your best knowledge to list all revevant concepts.
Each chapter should be about a specific prompting technique or its variation. Avoid adding evaluation methods or metrics for sections dedicated to specific prompting techniques.
Focus on the rules of the subject and how to use specific technique.
Try to fit number of proposals between 6 to 8 sections.
"""
]

content_instructs = """
Try to keep your answers between 300 and 1000 words.
Focus on providing all different variations of technique you are writing about. 
So for example, section about Cot should include all possible variations of Cot like Tabular CoT, multimodal Cot.
If it is a chapter about a specific technique, avoid writing about combinations with other techniques or applications in different fields.
Provide some examples where it fits specifics of a given topic.
Put your attention on providing detailed technical information. Avoid general paragraphs. Treat it like a manual to use something.
Avoid writing about evaluation methods or metrics when it is not a main subject of the section.
Focus explicitly on the subject of the section. Your main goal is to provide detailed information within provided list of concepts without going into other topics.
"""

In [76]:
model_def = {
"model_lib": {
    "lib": "langchain_google_genai",
    "module": "ChatGoogleGenerativeAI"
},
"model_args": {
    "model": "gemini-2.0-flash",
    "temperature": 0.5,
    "max_output_tokens": 2048,
    "max_tokens": 4096 
}
}

json_fix_config = {
    "prompt_path": "../prompt_templates/json_formatter_v01.json",
    }


general_config = {
    "main_subject": "Prompt Engineering Techniques",
    "add_concepts": True,
    "chapter_nesting_level": 2,
    "project_name": "pet_6",
    "sections_sep": '\n'
}

concepts_config = {
    "prompt_paths": ["../prompt_templates/concepts_v01a.json"],
    "concepts_num": 80,
    "instructs": [concepts_instructs, concept_instructs2],
    "inputs_mult": 1,
    "output_prefix": "To help you with this task, here is a list of concepts and subjects related to a book that you are writing: "
}

parts_config = {
    "prompt_paths": ["../prompt_templates/parts_v01a.json", "../prompt_templates/parts_v01b.json"],
    "instructs": parts_instructs,
    "add_concepts": True,
    "section_args": [
        {'key': 'title', 'format': str, 'fill': ''},
        {'key': 'subtitle', 'format': str, 'fill': ''},
        {'key': 'description', 'format': str, 'fill': ''},
        {'key': 'concepts', 'format': str, 'fill': ''}
    ]
}

chapter_config = {
    "prompt_paths": ["../prompt_templates/chapters_v01a.json"],
    "instructs": chapters_instructs,
    "levels_for_choice": 1,
    "section_args": [
        {'key': 'title', 'format': str, 'fill': ''},
        {'key': 'subtitle', 'format': str, 'fill': ''},
        {'key': 'description', 'format': str, 'fill': ''},
        {'key': 'concepts', 'format': str, 'fill': ''}
    ],
    "toc_keys": ['id', 'title', 'subtitle'],
    "toc_max_nesting": 1
}

content_config = {
    "prompt_path": "../prompt_templates/contents_v01.json",
    "instructs": content_instructs,
    "batch_size": 32,
    "toc_keys": ['id', 'title', 'subtitle'],
    "toc_max_nesting": 1
}

intro_config = {
    "prompt_path": "../prompt_templates/intro_v01.json",
}

In [78]:
guide_builder = GuideBuilder(
    model_def=model_def, 
    json_fix_config=json_fix_config, 
    concepts_config=concepts_config, 
    general_config=general_config,
    parts_config=parts_config,
    chapter_config=chapter_config,
    content_config=content_config,
    intro_config=intro_config
    )

In [30]:
guide_builder.sections = load_from_json('../locals/pet_5/contents/outputs-Intelligent-Human-2509.json')[-1]
guide_builder.outputs = load_from_json('../locals/pet_5/contents/outputs-Intelligent-Human-2509.json')

In [64]:
guide_builder.intro_path

In [65]:
guide_builder.gen_intro()

Generating Introduction...
Introduction generation completed.


In [ ]:
guide_builder.gen_parts()

In [38]:
guide_builder.load_outputs('contents/outputs-Amiable-Eland-2156.json')

In [80]:
print(len(guide_builder.sections))

243


In [81]:
guide_builder.gen_contents()

  0%|          | 0/8 [00:00<?, ?it/s]

In [37]:
guide_builder.content_path

'../locals/pet_5/contents/Bright-Cheetah-7196'

In [18]:
folder_path = 'contents/Fearless-Emu-5820'
count_files_in_folder(folder_path)

Number of files in the folder: 501


In [ ]:
folder_path = 'contents/Quick-Armadillo-1897'
count_files_in_folder(folder_path)


Number of files in the folder: 1797


In [3]:
outputs = load_from_json('../locals/pet/contents/outputs-Independent-Horse-7597.json')

In [32]:
toc = TableOfContents(guide_builder.outputs, '../locals/pet/contents/Fearless-Emu-5820')
toc.get_flat_sections(1, ['id', 'title', 'subtitle'])

['1 Part I: Foundational Prompting Techniques Establishing the Basis for Effective Prompt Engineering',
 '1.1 Prompt Anatomy: Deconstructing the Building Blocks of Effective Prompts Understanding the structure and key components that make up a well-formed prompt.',
 '1.2 Zero-Shot Prompting: Eliciting Responses Without Examples Harnessing the power of language models to perform tasks with no prior demonstrations.',
 '1.3 Input Context Engineering: Shaping the Information Landscape Strategically designing the input context to guide the model towards desired outcomes.',
 "1.4 Output Format Specification: Defining the Structure and Style of Responses Controlling the format, style, and structure of the language model's output.",
 "1.5 Constraint Definition: Guiding Generation with Boundaries and Limitations Implementing constraints within prompts to refine and control the language model's output.",
 '1.6 Prompt Templates and Libraries: Reusable Structures for Efficiency Creating and utiliz

# Content generation

In [85]:
main_part_template = '# **{title}**\n## {subtitle}\n'
sub_part_template = '## **{id} {title}:** {subtitle}\n'

In [86]:
pdf_args = {
    "project_name": "pet_6",
    "contents_path": "Powerful-Beaver-3805",
    "chapters_file": "outputs-Patient-Eel-1553.json",
    "intro_file": "intro-Fearless-Fly-3661.json",
    "title": "Prompt Engineering Techniques",
    "header_path": "../resources/header.txt",
    "footer_path": "../resources/footer.html",
    "toc_path": "../resources/toc.txt",
    "logo_path": "../resources/white_logo.png",
    "footer_logo_path": "../resources/resized_fzzyfoxx_logo.png",
    "main_part_template": main_part_template,
    "sub_part_template": sub_part_template,
    "formatting": {'font_size': 10, 'font': 'Tahoma', 'toc_title_font_size': 18, 'toc_font_size': 12},
    "cover_args": {'palette': 'viridis', 'border_color': 100, 'darkening_ratio': 90}
}

In [87]:
m2p = Markdown2PDF(**pdf_args, section_limit=None)

In [88]:
m2p._assign_contents()

In [89]:
m2p._write_chapters()

  0%|          | 0/49 [00:00<?, ?it/s]

In [90]:
m2p._generate_cover()

Generating Cover...


In [91]:
m2p._generate_intro()

Generating Introduction...


In [92]:
m2p._save_toc()

'../locals/pet_6/chapters/toc.pdf'

In [93]:
m2p._concat_pages()

Concatenating pages...


  0%|          | 0/49 [00:00<?, ?it/s]